# 1. Set up environment

The boring part!

Installs all required dependencies and spins up a local devnet that will run Nada programs

In [2]:
%pip install nada-ai~=0.3.0 --quiet


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import time
import sys
import uuid

In [5]:
# Configure telemetry settings
enable_telemetry = True  #@param {type:"boolean"}
my_identifier = "test"  #@param {type:"string"}

In [1]:
# Install the nilup tool and then use that to install the Nillion SDK
!curl https://nilup.nilogy.xyz/install.sh | bash

# # Update Path if ran in colab
# if "google.colab" in sys.modules:
#     os.environ["PATH"] += ":/root/.nilup/bin"
#     os.environ["PATH"] += ":/root/.nilup/sdks/latest/"

# Set telemetry if opted in
if enable_telemetry:
    identifier = f"nada-ai-spam-detection-{str(uuid.uuid4())}-{my_identifier}"
    !echo 'yes' | nilup instrumentation enable --wallet {identifier}

# Install the lastest SDK and initialise it
!nilup init
!nilup install latest
!nilup use latest

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7810  100  7810    0     0   6008      0  0:00:01  0:00:01 --:--:--  6012

nilup has been installed into /Users/ash/.nilup/bin.

$PATH is already up-to-date. You may begin using nilup now!



NameError: name 'sys' is not defined

In [1]:
# Spin up local Nillion devnet
get_ipython().system = os.system
!nohup nillion-devnet &

time.sleep(20)  # Wait for devnet

NameError: name 'os' is not defined

# 2. Build Nada program

In [3]:
!nada build

Building program: sentimental_analysis
Build complete!


In [ ]:
# You will see that the program was compiled in a .nada.bin file
!ls target | grep sentimental_analysis

sentimental_analysis.nada.bin


# 4. Provide model

Let's step into the shoes of the model provider.

We will train a spam detection model and upload the weights as secrets.

In [ ]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import zipfile

import joblib
import pandas as pd
import numpy as np

import requests

from dotenv import load_dotenv
from io import BytesIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

from config import DIM

home = os.getenv("HOME")
load_dotenv(f"{home}/sentimental_analysis/nillion-testnet.env")

True

In [ ]:
# Load the movie review Dataset
df = pd.read_csv('dataset/movie_reviews.csv', header=0, sep=",", names=["review", "sentiment"])

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
# Split data into features and labels
X = df["review"]
y = df["sentiment"].map({"positive": 1, "negative": 0}) # Convert labels to binary (1 for positive, 0 for negative)


# 3. Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Transform text to set of numerical features
vectorizer = TfidfVectorizer(max_features=DIM)  # Limiting to fixed set of features
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Save the vectorizer to a file
joblib.dump(vectorizer, "model/vectorizer.joblib")

['model/vectorizer.joblib']

In [12]:
# Train the logistic regression model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_vectorized, y_train)

print("Learned model coeffs are:", model.coef_)
print("Learned model intercept is:", model.intercept_)

Learned model coeffs are: [[ 0.21636137 -0.16730774  1.63125533 ... -0.87058316 -1.24181246
  -0.0494308 ]]
Learned model intercept is: [0.01389142]


In [13]:
# 6. Evaluate the model
y_pred = model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

Accuracy: 0.90

Classification Report:
              precision    recall  f1-score   support

    Negative       0.90      0.88      0.89      4961
    Positive       0.89      0.91      0.90      5039

    accuracy                           0.90     10000
   macro avg       0.90      0.89      0.89     10000
weighted avg       0.90      0.90      0.89     10000



In [14]:
# Save the classifier to a file
joblib.dump(model, "model/classifier.joblib")

['model/classifier.joblib']

In [15]:
!python 01_provide_model.py \
    --model-path model/classifier.joblib \
    --out-path target/identifiers.json

Importing plotly failed. Interactive plots will not work.
Storing program...
Getting quote for operation...
Traceback (most recent call last):
  File "/Users/ash/sentimental_analysis/01_provide_model.py", line 120, in <module>
    asyncio.run(main(ARGS.model_path, ARGS.out_path))
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/runners.py", line 194, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/base_events.py", line 687, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/Users/ash/sentimental_analysis/01_provide_model.py", line 

# 5. Provide input and run inference

Now that the model has been provided, we can step into the shoes of the model user.

We will provide an input to the program and run the model on it.

In [56]:
vectorizer: TfidfVectorizer = joblib.load("model/vectorizer.joblib")

In [71]:
# Let's find out whether it's a billion dollar opportunity or pyramid scheme
INPUT_DATA = "this movie is a awesome, I just loved the actors and the suppoting cast I feel more movies like this should be made"

[features] = vectorizer.transform([INPUT_DATA]).toarray().tolist()

In [72]:
features = np.array(features).astype(float)
print(np);
np.save("model/features.npy", features)

<module 'numpy' from '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/__init__.py'>


In [73]:
!python 02_run_inference.py \
    --features-path model/features.npy \
    --in-path target/identifiers.json

Storing input data...
Getting quote for operation...
Quote cost is 48002 unil


Traceback (most recent call last):
  File "/Users/ash/sentimental_analysis/02_run_inference.py", line 140, in <module>
    asyncio.run(main(ARGS.features_path, ARGS.in_path))
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/runners.py", line 194, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/base_events.py", line 687, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/Users/ash/sentimental_analysis/02_run_inference.py", line 87, in main
    features_store_id = await store_secret_array(
                        ^^^^^^^^^^^^^^^^^^^^

256

In [27]:
# Let's sense-check these results versus what we would have gotten in plain-text
vectorizer: TfidfVectorizer = joblib.load("model/vectorizer.joblib")
classifier: LogisticRegression = joblib.load("model/classifier.joblib")
features = vectorizer.transform([INPUT_DATA]).toarray().tolist()

[logit_plain_text] = classifier.decision_function(features)
probabilities = 1 / (1 + np.exp(-logit_plain_text))
sentiment = "positive" if probabilities > 0.5 else "negative"
print("Logit in plain text: {}".format(logit_plain_text))

output_probability_plain_text = classifier.predict_proba(features)[0][1]
print(
    "Probability of spam in plain text: {:.6f}%".format(
        output_probability_plain_text * 100
    )
)

Logit in plain text: -0.9943009892551997
Probability of spam in plain text: 27.006339%
